# Writing an object class called 'Bond'
## Main Idea
I want to create a class object so that I can give a few values to define a bond and instantly obtain the one missing value. This means I different constructor methods. Stack Overflow says that I cannot use several __init__ methods with different signatures/lists of arguments in Python, so instead I write classmethods with different names that specify the way the Bond is being defined.

In [35]:
import numpy as np

class Bond:
    
    def __init__(self, maturity: int, rate: float, coupon: float, face: float = 1000., frequency: int = 2):
        self.maturity = maturity
        self.coupon = coupon
        self.rate = rate
        self.face = face
        self.frequency = frequency
        
        self.cash_flows = self.create_flows(maturity, coupon, face, frequency)
        self.pv = self.present_value(rate,self.cash_flows)
        
    @classmethod
    def no_pv(cls, maturity: int, rate: float, coupon: float, face: float = 1000., frequency: int = 2):
        return cls(maturity, rate, coupon, face, frequency)
    
    @classmethod
    def no_ytm(cls, maturity: int, pv: float, coupon: float, face: float = 1000., frequency: int = 2):
        
        cls.frequency = frequency
        rate = cls.ytm(cls,pv,cls.create_flows(cls, maturity, coupon, face, frequency))
        
        return cls(maturity, rate, coupon, face, frequency)
    
    @classmethod
    def no_coupon(cls, maturity: int, pv: float, rate: float, face: float = 1000., frequency: int = 2):
        
        cls.frequency = frequency
        coupon = cls.coupon_rate(cls,maturity,pv,rate,face,frequency)
        
        return cls(maturity, rate, coupon, face, frequency)
    
    @classmethod
    def no_face(cls, maturity: int, pv: float, rate: float, coupon: float, frequency: int = 2):
    
        cls.frequency = frequency
        face = cls.face_value(cls,maturity,pv,rate,coupon,frequency)
        
        return cls(maturity, rate, coupon, face, frequency)
    
    def __str__(self):
        return f'a $%.02f, %.02f percent coupon bond with %d years to maturity' % (self.pv,self.coupon*100,self.maturity)

    def create_flows(self, maturity: int, coupon: float, face: float, frequency: int) -> np.ndarray:
        pmt = coupon * face / frequency
        bond = np.full(maturity*frequency,pmt)
        bond[-1] += face
        return bond
            
    def present_value(self, rate: float, cash_flows: np.ndarray) -> float:
        pv = 0
        for count, flow in enumerate(cash_flows):
            pv += flow/((1+rate/self.frequency)**(count+1))
        return pv
    
    def ytm(self, pv: float, cash_flows: np.ndarray, lower: float = 0., upper: float = 1.) -> float:

        while(True): 
            guess = (lower + upper)/2
            #      Guessed price      -       Real price
            diff = self.present_value(self, guess, cash_flows) - pv
            if (abs(diff) < 0.00001):
                return guess
            else:
                if (diff > 0): # The guessed price is too high, so our guess is too low
                    lower = guess # Since our guess is too low, we raise the lower limit of our search
                else:
                    upper = guess
        
    def coupon_rate(self, maturity: int, pv: float, rate: float, face: float, frequency: int):
        
        factors = np.ones(maturity*frequency)
        coupon = (pv - face/(1+rate/frequency)**(maturity*frequency))
        coupon /= self.present_value(self,rate,factors)
        coupon /= face
        coupon *= frequency
        
        return coupon
    
    def face_value(self, maturity: int, pv: float, rate: float, coupon: float, frequency: int):
        
        coupons = self.create_flows(self,maturity,coupon,1,frequency)
        face = pv / self.present_value(self,rate,coupons)
        
        return face

In [36]:
b = Bond.no_face(maturity = 10, pv = 1000., rate = .05, coupon = .06)

In [37]:
print(str(b).capitalize())

A $1000.00, 6.00 percent coupon bond with 10 years to maturity


In [38]:
print(b.maturity,b.pv,b.rate,b.coupon,b.face,b.frequency,b.cash_flows)

10 1000.0 0.05 0.06 927.6904176426453 2 [ 27.83071253  27.83071253  27.83071253  27.83071253  27.83071253
  27.83071253  27.83071253  27.83071253  27.83071253  27.83071253
  27.83071253  27.83071253  27.83071253  27.83071253  27.83071253
  27.83071253  27.83071253  27.83071253  27.83071253 955.52113017]
